In [7]:
from pdf_downloader import download_pdf

from parse_pdf_pymupdf import pymupdf_parse_pdf_to_text as pymupdf_parser
from parse_pdf_marker import parse_pdf_with_marker as marker_parser

from config import test_config

pdf_url = test_config.PDF_URLS[2]
test_folder_name = 'test_1'

In [2]:
downloaded_pdf = download_pdf(pdf_url)

In [3]:
#parsed_pdf_pymupdf = pymupdf_parser(downloaded_pdf)

In [8]:
parsed_pdf_marker = marker_parser("/Users/nicolaszumarragafernandez/Documents/WORK/SEASTONE_AI/TEAMBID/CODE/nlp_pipeline_tests/clean_ai_tender_summaries/data/raw_pdfs/DOC_CN2024-001193585.pdf")

In [5]:
from markdown_chunker import langchain_chunk_markdown as chunker

chunks, chunk_files = chunker(parsed_pdf_marker,
                              output_dir=f'data/chunks/{test_folder_name}',
                              chunk_size=1024)


Created 344 chunks from data/markdown/PCAP Coor POB.md
Saved chunk index to data/chunks/test_1/PCAP Coor POB_index.json


In [6]:
from create_embeddings_fastembed import create_embeddings

embeddings, index_path = create_embeddings(
    chunks,
    output_dir=f"data/embeddings/{test_folder_name}",
    model_name="BAAI/bge-small-en-v1.5"
)

/Users/nicolaszumarragafernandez/Documents/WORK/SEASTONE_AI/TEAMBID/CODE/nlp_pipeline_tests/pdf_pipeline_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model: BAAI/bge-small-en-v1.5
Created 344 embeddings with model BAAI/bge-small-en-v1.5
Saved embeddings index to data/embeddings/test_1/embeddings_index.json


In [7]:
from simple_rag import SimpleRAG
import json

groq_rag = SimpleRAG(
    embeddings,
    model_name="llama3-70b-8192"
)

query = "¿Cuál es el plazo de cancelacion de la garantia efectiva?"
response = groq_rag.query(query)



print(json.dumps(response, indent=2, ensure_ascii=False))

Loading query embedding model: BAAI/bge-small-en-v1.5
Chunk content for RAG: 6.- GARANTÍAS EXIGIBLES.

La garantía no será devuelta o cancelada hasta que se haya producido el vencimiento del plazo de garantía establecido en la carátula de este pliego o, en su caso, el que conste en el contrato, y cumplido satisfactoriamente el contrato de que se trate, o hasta que se declare la resolución de éste sin culpa de la contratista. La tramitación se ajustará a las reglas establecidas en el artículo 111 de la LCSP.  
En el supuesto de recepción parcial la contratista sólo podrá solicitar la devolución o cancelación de la parte proporcional de la garantía cuando así se haya autorizado expresamente en la carátula de este pliego.  
En los casos de cesión de contratos no se procederá a la devolución o cancelación de la garantía prestada por la cedente hasta que se halle formalmente constituida la de la cesionaria.
Chunk content for RAG: 11.3.- Vicios y defectos durante el plazo de garantía.

El pl